# Prepare ACC data for analyses

## Information
Project: White Stork Affenberg, CareCenter, CASCB  
Author: Iris Bontekoe - parts of the code were provided by Andrea Flack (in R)  
Date Started: 13 July 2021  
Date Version 01: 13 July 2021  
Date last modified: 11 October 2021  

Program: Python 3.9

Description: This script loads the data and prepares it for the analyses. 

#### Version control
01: First version


#### Instructions
Make sure that the script is executed from top to bottom to ensure that everything works. Once a part of the script is executed and the files are saved, you can run the "Preparation" and then continue executing the next part without first executing the previous parts again.

## Preparation

In [3]:
# Import stuff that is necessary to execute the script
import os, glob
import pandas as pd
import datetime
#import pickle
#import time
import numpy as np
#import geopy.distance
#from collections import namedtuple
#from astral import LocationInfo
#from astral.sun import sun
from statistics import mean

# Set the path to the folder where the data is located
data_folder = "C:/Users/ibontekoe/Desktop/R_Analyses/DATA/"

## Prepare data

In [3]:
# Define a list of columns to keep
columns_to_keep = [
    "timestamp",
    "tag-local-identifier",
    "individual-local-identifier",
    "eobs:accelerations-raw",
    "sensor-type"
]

# Load the release-death data
ReleaseDeath = pd.read_csv(data_folder + "Release_Death.csv",sep=",",low_memory=False)

In [4]:
# Define function
# Write a function to make sure all objects are removed afterwards
def function_prep(Released):
    
    # Load the data
    data = pd.read_csv(data_folder+file_name_in,sep=",",low_memory=False)

    # Only keep columns that are necessary for further steps in the analyses
    data = data[columns_to_keep]

    # Only keep ACC data
    data = data[data["sensor-type"]=="acceleration"]

    # Convert the timestamps
    data["timestamp"] = pd.to_datetime(data["timestamp"],format="%Y-%m-%d %H:%M:%S.%f")

    # Remove data before release and after death
    
    # Add the timing of release and death to the data
    data["Release"] = pd.to_datetime(data["tag-local-identifier"].map(ReleaseDeath[ReleaseDeath["Aviary"].isin(aviary)].set_index("tag.local.identifier")["Release"].to_dict()),format="%d-%m-%Y %H:%M")
    data["Death"] = pd.to_datetime(data["tag-local-identifier"].map(ReleaseDeath[ReleaseDeath["Aviary"].isin(aviary)].set_index("tag.local.identifier")["Death"].to_dict()),format="%d-%m-%Y %H:%M")

    # Only keep data after release and before death
    data = data[(data["Release"].isna() | (data["timestamp"] >= data["Release"])) & (data["Death"].isna() | (data["timestamp"] <= data["Death"]))]
   
    # Sort the data
    data.sort_values(["tag-local-identifier","timestamp"], ascending=[True,True], inplace=True)

    #--- Split the ACC into colums X Y Z ---#
    
    # Make empty list to store the values in
    timestamp = []
    identifier = []
    X_raw = []
    Y_raw = []
    Z_raw = []
    ACC_num = []
    
    for ind in data["tag-local-identifier"].unique():
        timestamp__ = []
        identifier__ = []
        X_raw__ = []
        Y_raw__ = []
        Z_raw__ = []
        ACC_num__ = []

        data2 = data[data["tag-local-identifier"]==ind]
        
        # Split the ACC values into columns
        for i in range(len(data2)): 

            # Split the ACC values
            SplittedACC = data2.iloc[i]["eobs:accelerations-raw"].split()
            timestamp_ = np.repeat(data2.iloc[i]["timestamp"],len(SplittedACC)/3)
            identifier_ = np.repeat(data2.iloc[i]["tag-local-identifier"],len(SplittedACC)/3)
            X_raw_ = SplittedACC[0:len(SplittedACC):3]
            Y_raw_ = SplittedACC[1:len(SplittedACC):3]
            Z_raw_ = SplittedACC[2:len(SplittedACC):3]
            ACC_num_ = [*range(0+1,round(len(SplittedACC)/3)+1)]

            timestamp__ += timestamp_.tolist()
            identifier__ += identifier_.tolist()
            X_raw__ += X_raw_
            Y_raw__ += Y_raw_
            Z_raw__ += Z_raw_
            ACC_num__ += ACC_num_
            
        timestamp += timestamp__
        identifier += identifier__
        X_raw += X_raw__
        Y_raw += Y_raw__
        Z_raw += Z_raw__
        ACC_num += ACC_num__

    # Combine the list into a data frame
    data = pd.DataFrame({"timestamp":timestamp,"tag-local-identifier":identifier,"X_raw":X_raw,"Y_raw":Y_raw,"Z_raw":Z_raw,"ACC_num": ACC_num})

    #--- Transform the values to g or m/s2 ---#
    
    # Calibrate ACC data # values from Andrea
    cal_xzero = 2042
    cal_cx = 0.0020
    cal_yzero = 2042
    cal_cy = 0.0020
    cal_zzero = 2049
    cal_cz = 0.0023
    cal_g = 9.80665

    # Convert ACC raw values to meaningful unit (m/s2)
    data["X_mps"] = [(int(i)-cal_xzero)*cal_cx*cal_g for i in data["X_raw"]]
    data["Y_mps"] = [(int(i)-cal_yzero)*cal_cy*cal_g for i in data["Y_raw"]]
    data["Z_mps"] = [(int(i)-cal_zzero)*cal_cz*cal_g for i in data["Z_raw"]]
    
    # Add the aviary
    data["Aviary"] = Aviary
    
    # Add an individual ID
    data["Individual"] = data["Aviary"].map(str)+"_"+data["tag-local-identifier"].map(str)
    
    # Save the data into a new csv or pkl file
    data.to_pickle(data_folder+file_name_out)

In [5]:
#=================#
#=== Affenberg ===#
#=================#

# Define objects
file_name_in = "White Stork Affenberg releases MPIAB_ACC.csv"
file_name_out = "DataAff_ACC.pkl"
aviary = ["Affenberg_2019","Affenberg_2020"]
Aviary = "Affenberg"

# Execute the function and print the run time
start = datetime.datetime.now()
function_prep(Released=True)
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

2021-10-11 10:24:35.630651
0:06:30.155377


In [6]:
#==================#
#=== CareCenter ===#
#==================#

# Define objects
file_name_in = "LifeTrack White Stork SW Germany Care Centre Releases_ACC.csv"
file_name_out = "DataCC_ACC.pkl"
aviary = ["CareCenter_2019","CareCenter_2020"]
Aviary = "CareCenter"

# Execute the function and print the run time
start = datetime.datetime.now()
function_prep(Released=True)
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

2021-10-11 10:26:19.160897
0:01:43.514324


In [7]:
#=============#
#=== CASCB ===#
#=============#

# Define objects
file_name_in = "LifeTrack White Stork SW Germany CASCB_ACC.csv"
file_name_out = "DataCASCB_ACC.pkl"
aviary = ["CASCB_East","CASCB_West"]
Aviary = "CASCB"

# Execute the function and print the run time
start = datetime.datetime.now()
function_prep(Released=False)
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

2021-10-11 10:30:23.587955
0:04:04.416914


## Calculate ODBA

In [8]:
# Define the function
def CalculateODBA():

    # Load data
    data = pd.read_pickle(data_folder+file_name_in)

    # Round the timestamp to minutes
    data["time"] = data["timestamp"].round('min')

    # Make empty list to store the values in
    Timestamp = []
    Tag = []
    ODBA = []
    Ind = []

    for tsp in data["time"].unique():
        
        # Make empty list to store the values in
        Timestamp_ = []
        Tag_ = []
        ODBA_ = []
        Ind_ = []

        data2 = data[data["time"]==tsp]

        for ind in data2["tag-local-identifier"].unique():

            data3 = data2[data2["tag-local-identifier"]==ind]

            # Calculate ODBA
            Diff_X = abs(data3["X_mps"] - mean(data3["X_mps"]))
            Diff_Y = abs(data3["Y_mps"] - mean(data3["Y_mps"]))
            Diff_Z = abs(data3["Z_mps"] - mean(data3["Z_mps"]))

            ODBA__ = mean(Diff_X)+mean(Diff_Y)+mean(Diff_Z)

            Timestamp__ = data3["timestamp"].unique()[0]
            Tag__ = ind
            Ind__ = data3["Individual"].unique()[0]

            # Put values into list
            Timestamp_ += [Timestamp__]
            Tag_ += [Tag__]
            ODBA_ += [ODBA__]
            Ind_ += [Ind__]

        # Put values into list
        Timestamp += Timestamp_
        Tag += Tag_
        ODBA += ODBA_
        Ind += Ind_     

    # Combine the lists into a dataframe
    data = pd.DataFrame({"timestamp": Timestamp,"tag-local-identifier": Tag,"Individual": Ind,"ODBA": ODBA})

    data["Aviary"] = Aviary
    data["Day"] = data["timestamp"].dt.date

    # Save the data
    data.to_pickle(data_folder+file_name_out)

In [9]:
#=================#
#=== Affenberg ===#
#=================#

# Define objects
file_name_in = "DataAff_ACC.pkl"
file_name_out = "DataAff_ODBA.pkl"
Aviary = "Affenberg"

# Execute the function and print the run time
start = datetime.datetime.now()
CalculateODBA()
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

2021-10-11 11:42:57.853055
0:56:14.152247


In [10]:
#==================#
#=== CareCenter ===#
#==================#

# Define objects
file_name_in = "DataCC_ACC.pkl"
file_name_out = "DataCC_ODBA.pkl"
Aviary = "CareCenter"

# Execute the function and print the run time
start = datetime.datetime.now()
CalculateODBA()
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

2021-10-11 12:00:14.915959
0:17:17.046102


In [11]:
#=============#
#=== CASCB ===#
#=============#

# Define objects
file_name_in = "DataCASCB_ACC.pkl"
file_name_out = "DataCASCB_ODBA.pkl"
Aviary = "CASCB"

# Execute the function and print the run time
start = datetime.datetime.now()
CalculateODBA()
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

2021-10-11 12:26:43.821850
0:26:28.894920


## Add weather variables, the location and flight information


- 

In [8]:
# Define the function nearest
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [9]:
# Define function
def AddVariables():

    # Load ODBA data
    data_O = pd.read_pickle(data_folder+file_name_in_ODBA)
    #data123=data_O

    # Load GPS data
    data_G = pd.read_pickle(data_folder+file_name_in_GPS)
    
    # Only keep data during flight (during a burst)
    #data_G = data_G[data_G["BelongsToBurst"]]
    #data_G = data_G[~data_G["FlyingID"].isna()]

    # Remove ACC data that is outside the subset of GPS data
    data_G["IndDay"] = data_G["tag-local-identifier"].map(str)+"_"+data_G["Day"].map(str)
    data_O["IndDay"] = data_O["tag-local-identifier"].map(str)+"_"+data_O["Day"].map(str)
    data_O = data_O[data_O["IndDay"].isin(data_G["IndDay"])]

    # Add a new column to data to enter Burst_A
    data_O["Burst_A"] = np.nan
    
    # Find the nearest start time for every burst and enter the start time in the new column
    data_O.reset_index(drop=True, inplace=True)
    data_O["UniqueNumber"] = range(len(data_O))

    for ind in data_G["tag-local-identifier"].unique():

        data_O_sub = data_O[data_O["tag-local-identifier"]==ind]
        data_G_sub = data_G[data_G["tag-local-identifier"]==ind]
        
        for row in data_O_sub["UniqueNumber"]:
            First_timestamp = data_O.loc[data_O["UniqueNumber"]==row,"timestamp"].unique()
            if (First_timestamp < (min(data_G_sub["timestamp"])-datetime.timedelta(minutes=10))):
                continue
                
            if (First_timestamp > (max(data_G_sub["timestamp"])+datetime.timedelta(minutes=10))):
                continue

            TimeSTPs = data_G_sub[data_G_sub["timestamp"]<=First_timestamp[0]]["timestamp"].unique()
           
            if len(TimeSTPs)<1:
                continue
                
            Nearest = nearest(items=TimeSTPs,pivot=First_timestamp)
            Nearest = data_G_sub.loc[data_G_sub["timestamp"]==Nearest,"Burst_A"].unique()
            
            if pd.isnull(Nearest):
                continue
                        
            data_O.loc[data_O["UniqueNumber"]==row,"Burst_A"] = Nearest

    data_O["DiffTime"]=abs((pd.to_datetime(data_O["Burst_A"]) - pd.to_datetime(data_O["timestamp"])).dt.total_seconds()/60)
    

    # Replace Burst_A by NA if the timedifference is more than 12 minutes
    data_O.loc[~(data_O["DiffTime"].isna()|(data_O["DiffTime"]<12)),"Burst_A"] = np.nan

    # Remove the data where Burst_A is NA
    data_O = data_O[~(data_O["Burst_A"].isna())]

    # Make a table with the information from the GPS data
    data_G2 = data_G.groupby(["Aviary","Day","tag-local-identifier","Individual","BurstID","Burst_A","IndDay"],as_index=False).agg(
        WindSupportPL_Mean = ("WindSupport_PL",'mean'),
        TempSL_Mean = ("Temp_SL",'mean'),
        WindSpeedPL_Mean = ("WindSpeed_PL",'mean'),
        TimestampG_End = ("timestamp",'last'),
        Lat_End = ("location-lat",'last'),
        Long_End = ("location-long",'last')
    )

    # Summarise the behaviour (flying, gliding, climbing) and weather at the end (last 5 seconds) of the burst
    data_G3 = data_G.groupby(["Aviary","Day","tag-local-identifier","Individual","BurstID","Burst_A","IndDay"],as_index=False).tail(5)
    data_G3 = data_G3.groupby(["Aviary","Day","tag-local-identifier","Individual","BurstID","Burst_A","IndDay"],as_index=False).agg(
        FlyingID = ("FlyingID",'mean'),
        GlidingID = ("GlidingID",'mean'),
        ClimbingID = ("ClimbingID",'mean'),
        Altitude_End = ("Altitude",'mean'),
        WindSupportPL_End = ("WindSupport_PL",'mean'),
        WindSpeedPL_End = ("WindSpeed_PL",'mean'),
        TempSL_End = ("Temp_SL",'mean'),
        #TimestampG_End = ("timestamp",'last'),
        ClimbingRate_End = ("ClimbingRate",'mean'),
        Sink_End = ("ClimbingRate",'mean'),
        GlidingSpeed_End = ("ground-speed",'mean'),
        GlidingAirspeed_End = ("AirSpeed_PL",'mean')
    )

    # Summarise flight properties during climbing bits
    data_G4 = data_G[~(data_G["ClimbingID"].isna())]
    data_G4 = data_G4.groupby(["Aviary","Day","tag-local-identifier","Individual","BurstID","Burst_A","IndDay"],as_index=False).agg(
        ClimbingRate_Mean = ("ClimbingRate",'mean')
    )

    # Summarise flight properties during climbing bits
    data_G5 = data_G[~(data_G["GlidingID"].isna())]
    data_G5 = data_G5.groupby(["Aviary","Day","tag-local-identifier","Individual","BurstID","Burst_A","IndDay"],as_index=False).agg(
        Sink_Mean = ("ClimbingRate",'mean'),
        GlidingSpeed_Mean = ("ground-speed",'mean'),
        GlidingAirspeed_Mean = ("AirSpeed_PL",'mean')
    )

    # Summarise the data for the last gliding segment of the burst (only if the gliding segment is at the end)
    data_G3g = data_G3[~(data_G3["GlidingID"].isna())]
    data_G3g = data_G3g[["Aviary","Day","tag-local-identifier","Individual","BurstID","Burst_A","IndDay","GlidingID"]]
    data_G["Burst_A"] = data_G.Burst_A.astype('datetime64[ns]')
    data_G3g = pd.merge(data_G,data_G3g)
    
    data_G3g = data_G3g.groupby(["Aviary","Day","tag-local-identifier","Individual","BurstID","Burst_A","IndDay","GlidingID"],as_index=False).agg(
        Sink_Segm = ("ClimbingRate",'mean'),
        GlidingSpeed_Segm = ("ground-speed",'mean'),
        GlidingAirspeed_Segm = ("AirSpeed_PL",'mean'),
        WindSupportPL_Segm = ("WindSupport_PL",'mean')
    )
    
    # Summarise the data for the last climbing segment of the burst (only if the climbing segment is at the end)
    data_G3c = data_G3[~(data_G3["ClimbingID"].isna())]
    data_G3c = data_G3c[["Aviary","Day","tag-local-identifier","Individual","BurstID","Burst_A","IndDay","ClimbingID"]]
    data_G3c = pd.merge(data_G,data_G3c)
    
    data_G3c = data_G3c.groupby(["Aviary","Day","tag-local-identifier","Individual","BurstID","Burst_A","IndDay","ClimbingID"],as_index=False).agg(
        ClimbingRate_Segm = ("ClimbingRate",'mean')
    )

    # Add the information to the ODBA data
    data_O["Burst_A"] = data_O.Burst_A.astype('datetime64[ns]')
    #data_G2["Burst_A"] = data_G2.Burst_A.astype('datetime64[ns]')
    #data_O.Day.astype('datetime64[ns]')
    #data_G2.Day.astype('datetime64[ns]')

    data_O2 = pd.merge(data_O,data_G2,on=["IndDay","Burst_A","tag-local-identifier","Aviary","Day","Individual"],how="outer")

    data_O2 = pd.merge(data_O2,data_G3,on=["IndDay","Burst_A","tag-local-identifier","Aviary","Day","Individual","BurstID"],how="outer")
    data_O2 = pd.merge(data_O2,data_G4,on=["IndDay","Burst_A","tag-local-identifier","Aviary","Day","Individual","BurstID"],how="outer")
    data_O2 = pd.merge(data_O2,data_G5,on=["IndDay","Burst_A","tag-local-identifier","Aviary","Day","Individual","BurstID"],how="outer")
    data_O2 = pd.merge(data_O2,data_G3g,on=["IndDay","Burst_A","tag-local-identifier","Aviary","Day","Individual","BurstID","GlidingID"],how="outer")
    data_O2 = pd.merge(data_O2,data_G3c,on=["IndDay","Burst_A","tag-local-identifier","Aviary","Day","Individual","BurstID","ClimbingID"],how="outer")

    data_O2["DiffTime_"] = (data_O2["timestamp"] - data_O2["TimestampG_End"]).dt.total_seconds()

    # Remove data that does not fit to a burst
    data_O2 = data_O2[~(data_O2["DiffTime_"].isna())]
    data_O2 = data_O2[data_O2["DiffTime_"]>=0]
    data_O2 = data_O2[data_O2["DiffTime_"]<120]
    
    # Add a column for climbing vs gliding
    data_O2["FlightType"] = np.nan
    data_O2.loc[~(data_O2["GlidingID"].isna()),"FlightType"] = "Gliding"
    data_O2.loc[~(data_O2["ClimbingID"].isna()),"FlightType"] = "Climbing"

    #-----------------#
    #- Save the data -#
    #-----------------#
    # Save the data into a new pkl file
    data_O2.to_pickle(data_folder+file_name_out)

In [10]:
#=================#
#=== Affenberg ===#
#=================#

# Define objects
file_name_in_ODBA = "DataAff_ODBA.pkl"
file_name_in_GPS = "DataAff_TempWind_S.pkl"
file_name_out = "DataAff_ODBA_Temp.pkl"

# Execute the function and print the run time
start = datetime.datetime.now()
AddVariables()
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

2022-03-18 16:11:06.387350
0:04:05.148521


In [11]:
#==================#
#=== CareCenter ===#
#==================#

# Define objects
file_name_in_ODBA = "DataCC_ODBA.pkl"
file_name_in_GPS = "DataCC_TempWind_S.pkl"
file_name_out = "DataCC_ODBA_Temp.pkl"

# Execute the function and print the run time
start = datetime.datetime.now()
AddVariables()
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

2022-03-18 16:29:49.350055
0:03:35.696099


In [12]:
#=============#
#=== CASCB ===#
#=============#

# Define objects
file_name_in_ODBA = "DataCASCB_ODBA.pkl"
file_name_in_GPS = "DataCASCB_TempWind_S.pkl"
file_name_out = "DataCASCB_ODBA_Temp.pkl"

# Execute the function and print the run time
start = datetime.datetime.now()
AddVariables()
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

2022-03-18 16:42:34.271731
0:12:44.889233


## Combine the files

In [13]:
# Combine the data for the different studies
def CombineAll(pattern):

    # Find all file names
    all_files = glob.glob(data_folder+"Data"+"*"+pattern+".pkl")

    # Load all data files for Affenberg
    data_all = (pd.read_pickle(f) for f in all_files)

    # Merge the files together
    data = pd.concat(data_all)

    # Save the data into a new csv file
    data.to_csv(data_folder+"All"+pattern+".csv")

In [14]:
# Execute the function and print the run time
start = datetime.datetime.now()
CombineAll(pattern="_ODBA_Temp")
print(datetime.datetime.now())
print(datetime.datetime.now()-start)

2022-03-18 16:42:34.467889
0:00:00.162852
